In [48]:
from matplotlib import pyplot as plt
import glob
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from scipy import interpolate
import pandas as pd

In [49]:
path = os.path.abspath(os.getcwd())

In [50]:
files = glob.glob(path+'/*simulation*[!mirrored]/*.dat') # Get all .dat files in the folders that are not marked as mirrored

In [51]:
# Filters all the repeating file paths with the same IC out
files_filtered = []
for file in files:
    found = False
    for file_f in files_filtered:
        start = file.find('/q_')
        if file[start:] in file_f:
            found = True
    if not found:
        files_filtered.append(file)

In [52]:
print(len(files),len(files_filtered))

2041 1370


In [53]:
data = np.loadtxt(files_filtered[0])
# Remove repeated times
mask = np.ones(len(data), dtype=bool)
last_time = -1
for i in range(len(data)):
    actual_time = data[i,0]
    if last_time >= actual_time:
        mask[i] = False
    last_time = actual_time
data = data[mask]

In [54]:
# 
samples = 1000
t = np.linspace(data[0,0],data[-1,0],samples)
data_resampled = np.zeros((len(t),len(data[0])))
data_resampled[:,0] = t
for i in range(1,len(data[0])):
    tck = interpolate.splrep(data[:,0],data[:,i], s=0, k=3)
    data_resampled[:,i] = interpolate.splev(t, tck, der=0)


In [55]:
# create the negative trajectory
data_inv = data_resampled.copy()
data_inv[:,1:] *= -1

In [56]:
# save the resulting np.array as a csv file
# Create path names
file = files_filtered[0]
path_to_save_gen = path + '/data_augmented_db' + file[file.find('/q_'):-4]
path_to_save = path_to_save_gen + '.txt'
path_to_save_mirrored = path_to_save_gen + '_mirrored.txt'

# saves np.arrays as .csv files
np.savetxt(path_to_save, data_resampled, delimiter=',')
np.savetxt(path_to_save_mirrored, data_inv, delimiter=',')

In [57]:
# %matplotlib qt
# for i in range(1,6):
#     plt.figure()
#     plt.plot(data[:,0],data[:,i],'x')
#     plt.plot(data_resampled[:,0],data_resampled[:,i])
#     plt.plot(data_inv[:,0],data_inv[:,i])

# Create augmented database

In [58]:
for file in files_filtered:
    data = np.loadtxt(file)
    # Remove repeated times
    mask = np.ones(len(data), dtype=bool)
    last_time = -1
    for i in range(len(data)):
        actual_time = data[i,0]
        if last_time >= actual_time:
            mask[i] = False
        last_time = actual_time
    data = data[mask]
    # Resample the data
    samples = 1000
    t = np.linspace(data[0,0],data[-1,0],samples)
    data_resampled = np.zeros((len(t),len(data[0])))
    data_resampled[:,0] = t
    for i in range(1,len(data[0])):
        tck = interpolate.splrep(data[:,0],data[:,i], s=0, k=3)
        data_resampled[:,i] = interpolate.splev(t, tck, der=0)
    # create the negative trajectory
    data_inv = data_resampled.copy()
    data_inv[:,1:] *= -1
    # save the resulting np.array as a csv file
    # Create path names
    path_to_save_gen = path + '/data_augmented_db' + file[file.find('/q_'):-4]
    path_to_save = path_to_save_gen + '.txt'
    path_to_save_mirrored = path_to_save_gen + '_mirrored.txt'

    # saves np.arrays as .csv files
    np.savetxt(path_to_save, data_resampled, delimiter=',')
    np.savetxt(path_to_save_mirrored, data_inv, delimiter=',')